In [13]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.robotparser import RobotFileParser
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import re

headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36"}

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
#     method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
    else: #bot 접근 못하게 막은데있어어 else뺌
        try:
            resp = requests.request(method, url,
                   params=params if method=='GET' else {},
                   data=params if method=='POST' else {},
                   headers=headers)
            resp.raise_for_status()
        except HTTPError as e:
            if limit > 0 and e.response.status_code >= 500: # 500 에러 재귀 ;너무많은 트래픽 안생기도록
                print(limit)
                time.sleep(300) # => random, 1초에 1번
                resp = download(url, params, headers, method, limit-1)
            else:
                print('[{}] '.format(e.response.status_code) + url)
                print(e.response.status_code)
                print(e.response.reason)
                print(e.response.headers)
    return resp

In [8]:
startrow =1
endrow = 1000
api_key = 'e8611c2a89e148f8a8f9'
url = 'http://openapi.foodsafetykorea.go.kr/api/' + api_key + '/I2790/json/' + str(startrow) + '/' + str(endrow)
resp = download(url, headers = headers,method= 'GET')
result = resp.json()
total = int(result['I2790']['total_count'])
de = total%1000

import sqlite3
con = sqlite3.connect('ex.db')
cur = con.cursor()

cur.executescript('''
    DROP TABLE IF EXISTS factor;
    CREATE TABLE factor(
        NUM           INTEGER NOT NULL,
        DESC_KOR      TEXT NOT NULL ,
        FOOD_GROUP    TEXT ,
        FOOD_CD      TEXT ,
        NUTR_CONT1     FLOAT DEFAULT 0.0,
        NUTR_CONT2     FLOAT DEFAULT 0.0,
        NUTR_CONT3     FLOAT DEFAULT 0.0,
        NUTR_CONT4   FLOAT DEFAULT 0.0,
        NUTR_CONT5   FLOAT DEFAULT 0.0,
        NUTR_CONT6   FLOAT DEFAULT 0.0,
        NUTR_CONT7   FLOAT DEFAULT 0.0,
        NUTR_CONT8   FLOAT DEFAULT 0.0,
        NUTR_CONT9   FLOAT DEFAULT 0.0,
        SERVING_SIZE   INTERGER ,
        MAKER_NAME    TEXT ,
        BGN_YEAR      TEXT ,
        SAMPLING_MONTH_NAME     TEXT ,
        SUB_REF_NAME      TEXT,
        SAMPLING_REGION_NAME   TEXT ,
        GROUP_NAME       TEXT ,
        RESEARCH_YEAR INTERGER DEFAULT 0,
        SAMPLING_REGION_CD  INTERGER DEFAULT 0,
        SAMPLING_MONTH_CD  TEXT ,
        ANIMAL_PLANT    TEXT 
    );
''')

In [9]:
for _ in result['I2790']['row']:
    cur.execute('''INSERT INTO factor(NUM,DESC_KOR ,
        FOOD_GROUP ,FOOD_CD ,NUTR_CONT1 ,
        NUTR_CONT2, NUTR_CONT3, NUTR_CONT4, NUTR_CONT5,
        NUTR_CONT6, NUTR_CONT7,  NUTR_CONT8, NUTR_CONT9,
        SERVING_SIZE  ,MAKER_NAME , BGN_YEAR, SAMPLING_MONTH_NAME  ,
        SUB_REF_NAME, SAMPLING_REGION_NAME, GROUP_NAME  ,
        RESEARCH_YEAR, SAMPLING_REGION_CD ,SAMPLING_MONTH_CD, ANIMAL_PLANT) 
        VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''',[
        _['NUM'],_['DESC_KOR'] ,
        _['FOOD_GROUP'] ,_['FOOD_CD'] ,_['NUTR_CONT1'] ,
        _['NUTR_CONT2']  ,_['NUTR_CONT3']  ,_['NUTR_CONT4']  , _['NUTR_CONT5'],
        _['NUTR_CONT6']   ,_['NUTR_CONT7'],  _['NUTR_CONT8'], _['NUTR_CONT9'],
        _['SERVING_SIZE']  ,_['MAKER_NAME'] , _['BGN_YEAR']     ,_['SAMPLING_MONTH_NAME']  ,
        _['SUB_REF_NAME'] , _['SAMPLING_REGION_NAME'] , _['GROUP_NAME']  ,
        _['RESEARCH_YEAR'] ,_['SAMPLING_REGION_CD'] ,_['SAMPLING_MONTH_CD'],_['ANIMAL_PLANT']])
con.commit()

In [10]:
for i in range(int(total/1000)):
    print("Count : ",i)
    startrow+= 1000
    if i==int(total/1000)-1:
        endrow += de
    else:
        endrow += 1000
    url = 'http://openapi.foodsafetykorea.go.kr/api/' + api_key + '/I2790/json/' + str(startrow) + '/' + str(endrow)
    resp = download(url, headers = headers,method= 'GET')
    result = resp.json()
    
    for _ in result['I2790']['row']:
        cur.execute('''INSERT INTO factor(NUM,DESC_KOR  ,
            FOOD_GROUP ,FOOD_CD ,NUTR_CONT1 ,
            NUTR_CONT2, NUTR_CONT3, NUTR_CONT4, NUTR_CONT5,
            NUTR_CONT6, NUTR_CONT7,  NUTR_CONT8, NUTR_CONT9,
            SERVING_SIZE  ,MAKER_NAME , BGN_YEAR, SAMPLING_MONTH_NAME  ,
            SUB_REF_NAME, SAMPLING_REGION_NAME, GROUP_NAME  ,
            RESEARCH_YEAR, SAMPLING_REGION_CD ,SAMPLING_MONTH_CD, ANIMAL_PLANT) 
            VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''',[
            _['NUM'],_['DESC_KOR'] ,
            _['FOOD_GROUP'] ,_['FOOD_CD'] ,_['NUTR_CONT1'] ,
            _['NUTR_CONT2']  ,_['NUTR_CONT3']  ,_['NUTR_CONT4']  , _['NUTR_CONT5'],
            _['NUTR_CONT6']   ,_['NUTR_CONT7'],  _['NUTR_CONT8'], _['NUTR_CONT9'],
            _['SERVING_SIZE']  ,_['MAKER_NAME'] , _['BGN_YEAR']     ,_['SAMPLING_MONTH_NAME']  ,
            _['SUB_REF_NAME'] , _['SAMPLING_REGION_NAME'] , _['GROUP_NAME']  ,
            _['RESEARCH_YEAR'] ,_['SAMPLING_REGION_CD'] ,_['SAMPLING_MONTH_CD'],_['ANIMAL_PLANT']])
    time.sleep(1)
    con.commit()
    time.sleep(1)

Count :  0
Count :  1
Count :  2
Count :  3
Count :  4
Count :  5
Count :  6
Count :  7
Count :  8
Count :  9
Count :  10
Count :  11
Count :  12
Count :  13
Count :  14
Count :  15
Count :  16
Count :  17
Count :  18
Count :  19
Count :  20
Count :  21
Count :  22
Count :  23
Count :  24
Count :  25
Count :  26
Count :  27
Count :  28


In [11]:
cur.close()
con.close()

# 음식 사진 모으기

### 대표 사진

In [44]:
import urllib
from selenium import webdriver
driver = webdriver.Chrome('/Users/sohyun/Downloads/chromedriver')
driver.get('http://xn--hg3bqa693eb6a.com/product/list.html?cate_no=24')

page = 1
url = 'http://xn--hg3bqa693eb6a.com/product/list.html?cate_no=24&page='
driver.get(url+str(page))
# driver.execute_script('console.log("hi")')
leng = len(driver.find_elements_by_class_name('other'))

In [45]:
for r in range(leng+1):    
    for i, _ in enumerate(driver.find_elements_by_css_selector('img')):
        if _.get_attribute('src').split('/')[4]=='product':
            urllib.request.urlretrieve(_.get_attribute('src'),"./img/" + str(r) + 'bakery' +str(i)+'.jpg')
    if r == leng:
        break
    page+=1
    driver.get(url+str(page))
    time.sleep(2)

In [49]:
driver.close()

In [14]:
download('https://www.panhonesta.com/menu', headers = headers, method='GET')

<Response [200]>

### 작은 이미지 추가

In [50]:
import urllib
from selenium import webdriver
driver = webdriver.Chrome('/Users/sohyun/Downloads/chromedriver')
driver.get('http://xn--hg3bqa693eb6a.com/product/%EC%95%BC%EB%AF%B8%EC%9A%94%EB%B0%80-8%E7%84%A1-%ED%98%84%EB%AF%B8-%EC%83%9D%EC%8C%80%EC%8B%9D%EB%B9%B5-%EC%8C%80%EB%B9%B5%EB%B9%84%EA%B1%B4%EB%B9%B5%EB%B9%84%EA%B1%B4%EC%8C%80%EB%B9%B5%EA%B1%B4%EA%B0%95%EB%B9%B5%EB%8B%A4%EC%9D%B4%EC%96%B4%ED%8A%B8%EB%B9%B5%EB%B9%84%EA%B1%B4/29/category/24/display/1/')

In [51]:
for i, _ in enumerate(driver.find_elements_by_class_name('ThumbImage')):
#     print(_.get_attribute('src'))
#     print(_)
    urllib.request.urlretrieve(_.get_attribute('src'),"./img/" + str(r) + 'bakery' +str(i)+'.jpg')


In [53]:
driver.close()

### 다음 페이지로 넘어가는 기능 추가

In [1]:
dic = {}
dic [1] = 3

In [2]:
dic

{1: 3}

In [3]:
dic[1] = 4

In [4]:
dic

{1: 4}